In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graph

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-24 19:21:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-02-24 19:21:58 (10.4 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read watch reviews 
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep=r'\t', header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [5]:
# Count Rows in Dataframe
df.count()

960872

In [6]:
# Build reviews dataframe
columns = ["review_id","customer_id","product_id","product_parent","review_date"]
reviews_df = df.select(columns)
reviews_df = reviews_df.withColumn("customer_id", reviews_df["customer_id"].cast(IntegerType()))
reviews_df = reviews_df.withColumn("product_parent", reviews_df["product_parent"].cast(IntegerType()))
reviews_df = reviews_df.withColumn("review_date",to_date(col("review_date"),"yyyy-mm-dd"))
reviews_df.show()
reviews_df.count()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370| 2015-01-31|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722| 2015-01-31|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390| 2015-01-31|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625| 2015-01-31|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221| 2015-01-31|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695| 2015-01-31|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279| 2015-01-31|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113| 2015-01-31|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|     263720892| 2015-01-31|
| RX27XIIWY5JPB|   44751341|B004LBPB7Q|     124278407| 2015-01-31|
|R15C7QEZT0LGZN|    9962330|B00KGTVGKS|      28017857| 2015-01-31|
|R361XSS37V0NCZ|   16097204|B0039UT5OU|     685450910| 2015-01

960872

In [7]:
reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [8]:
# Build products dataframe
columns = ["product_id","product_title"]
products_df = df.select(columns)
products_df = products_df.dropDuplicates()
products_df.show()
products_df.count()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EVX7V1I|Game Time Women's...|
|B009S4DODY|XOXO Women's XO55...|
|B00LBKXQRW|Anne Klein Women'...|
|B0009P679Y|Invicta Men's 993...|
|B00DHF30RU|M&c Women's | Cla...|
|B00NIDA43Y|GuTe Classic Skel...|
|B008EQDDPQ|Nautica Men's N13...|
|B004VRBZ66|Timex Men's T2N63...|
|B009BEO81I|        Fossil Riley|
|B008B39MTI|XOXO Women's XO55...|
|B00TGPM8PU|Handmade Wooden W...|
|B00VNXQQQ0|Eterna 2520-41-64...|
|B00B1PV1C4|Nautica Men's N19...|
|B00N1Y8TQ4|Tissot Men's T095...|
|B00G6DBTY6|red line Men's RL...|
|B00HM04AYI|Columbia Men's Fi...|
|B00VI8HB96|GUESS I90176L1 Wo...|
|B00IT25WJU|LanTac DGN556B Dr...|
|B0106S12XE|Skmei S Shock Ana...|
|B00FPSJ63Y|Michael Kors Ladi...|
+----------+--------------------+
only showing top 20 rows



149705

In [9]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [10]:
# Build customers dataframe
customers_df = df.groupby("customer_id").count()
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df = customers_df.withColumn("customer_id", customers_df["customer_id"].cast(IntegerType()))
customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))
customers_df.show()
customers_df.count()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11234247|             1|
|   38676202|             1|
|   51245251|             1|
|   52523043|             1|
|   45717823|            13|
|   10828210|             6|
|   12854219|             1|
|   51715942|             1|
|   11477755|             2|
|   20024088|             3|
|    1486564|             1|
|    8648812|             1|
|    1619706|             1|
|   25622063|             2|
|   16435476|             8|
|    8062860|             1|
|    5632722|             2|
|   44537075|             1|
|   21072459|             1|
|   15367495|             1|
+-----------+--------------+
only showing top 20 rows



719914

In [11]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [12]:
# Configure settings for RDS
# Note: To test this code, replace the following:
# <rds connection string/url>
# <rds db name>
# <userid>
# <password>
mode = "append"
jdbc_url="jdbc:postgresql://<rds connection string/url>:5432/<rds db name>"
config = {"user":"<userid>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [13]:
# write customers to table
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [14]:
# Write products to table
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [15]:
# Write reviews to table
reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)